## 現在のファイル状態の確認

In [ ]:
# import pandas as pd
# import os
# from mymodule import save_bin,read_bin,directory_to_dict

# Xeno2sound_ids = read_bin("saved_data/Xeno2sound_ids_all.bin")

# directory_path = 'E:\\xeno_data'
# result = directory_to_dict(directory_path)

In [ ]:
# df = pd.read_csv("saved_data/id2Xeno.csv",header = None)# 順番通りに取得する必要があるため
# the_ids = list(df[0])[1:]
# xeno_canto_id = list(df[1])[1:]

In [ ]:
# Wiki2Xeno = dict()
# for k,v in zip(the_ids,xeno_canto_id):
#     Wiki2Xeno[k] = v

In [ ]:
# # #ファイルが一つもないディレクトリを確認
# nodata_files = []
# for the_id in the_ids:
#     if the_id in result and result[the_id] == []:
#         nodata_files.append(the_id)

# #ファイルが欠損したディレクトリ（101未満かつ本来のファイル数に満たない）
# lack_files = []
# lack_files_num = []
# for the_id in the_ids:
#     if the_id in result and the_id in Xeno2sound_ids:
#         if result[the_id] != []:
#             if len(result[the_id])>=101:
#                 pass
#             else:
#                 d = len(Xeno2sound_ids[the_id])-len(result[the_id])
#                 if d!=0:
#                     lack_files.append(the_id)
#                     lack_files_num.append(d)

# # そもそも存在しないディレクトリ
# not_in_Xeno2sound_ids_files = []
# for the_id in the_ids:
#     if the_id in result:
#         if the_id not in Xeno2sound_ids:
#             not_in_Xeno2sound_ids_files.append(the_id)


# print("nodata files:",len(nodata_files))
# print("lack_files:",len(lack_files))
# print("not_in_Xeno2sound_ids_files:",len(not_in_Xeno2sound_ids_files))

nodata files: 378
lack_files: 0
not_in_Xeno2sound_ids_files: 0


## 学習用parquetデータの作成

In [3]:
from mymodule import save_bin,read_bin,directory_to_dict
import os

# ディレクトリを辞書に変換
result = directory_to_dict('E:\\xeno_data')
trimmed_result = directory_to_dict('E:\\trimmed_xeno_data')

# ディレクトリ辞書をタプルに変換
xeno_data_tuple = set()
for bird_id,sound_ids in result.items():
    for sound_id in sound_ids:
        xeno_data_tuple.add((bird_id,sound_id))

trimmed_xeno_data_tuple = set()
for bird_id,sound_ids in trimmed_result.items():
    for sound_id in sound_ids:
        trimmed_xeno_data_tuple.add((bird_id,sound_id))

In [4]:
print(len(trimmed_xeno_data_tuple),"/",len(xeno_data_tuple))
# 1135 / 337644

1135 / 337644


In [5]:
target_set = xeno_data_tuple - trimmed_xeno_data_tuple

## 無音域削除，ファイル長統一

In [4]:
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
from pydub import AudioSegment
from pydub.silence import split_on_silence

def process_audio(input_tuple):
    try:
        bird_id, sound_id = input_tuple
        sound = AudioSegment.from_mp3("E:\\xeno_data\\"+bird_id+"\\"+sound_id)
        chunks = split_on_silence(
            sound,
            min_silence_len=500,  # 0.5秒以上の無音を検出
            silence_thresh=-90    # -90dBFS 以下を無音と判定
        )

        combined = AudioSegment.empty()
        for chunk in chunks:
            combined += chunk

        if len(combined) > 30_000:
            combined = combined[:30_000]
        
        if not os.path.exists("E:\\trimmed_xeno_data\\"+bird_id):
            os.makedirs("E:\\trimmed_xeno_data\\"+bird_id)
            
        combined.export("E:\\trimmed_xeno_data\\"+bird_id+"\\"+sound_id, format="mp3")

    except:
        return input_tuple

In [ ]:
from multiprocessing import Pool

pool_size = 10

with Pool(pool_size) as p:
    results = p.map(process_audio, target_set)

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

# i = 0
i = 71
error = []

for bird_id,sound_ids in result.items():#len(result):9684
    print('\r%d / %d' %(i, len(result)), end='')
    try:
        if os.path.exists("E:\\xeno_data\\"+bird_id):
            for sound_id in sound_ids:
                sound = AudioSegment.from_mp3("E:\\xeno_data\\"+bird_id+"\\"+sound_id)
                chunks = split_on_silence(
                    sound,
                    min_silence_len=500,  # 0.5秒以上の無音を検出
                    silence_thresh=-90    # -90dBFS 以下を無音と判定
                )

                combined = AudioSegment.empty()
                for chunk in chunks:
                    combined += chunk

                if len(combined) > 30_000:
                    combined = combined[:30_000]
                
                if not os.path.exists("E:\\trimmed_xeno_data\\"+bird_id):
                    os.makedirs("E:\\trimmed_xeno_data\\"+bird_id)
                    
                combined.export("E:\\trimmed_xeno_data\\"+bird_id+"\\"+sound_id, format="mp3")
    except:
        error.append((bird_id,sound_id))
    i += 1

117 / 9683

In [7]:
i

79

In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

# def process_audio(file_path, silence_thresh=-40, min_silence_len=1000, compress_time=30000):
#     audio = AudioSegment.from_mp3(file_path)
#     nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
#     non_silent_audio = AudioSegment.empty()
#     compressed_audio = non_silent_audio[:compress_time]
#     return compressed_audio.export(format="wav").read()

def process_audio(input_file, duration=60_000):
    audio = AudioSegment.from_file(input_file)
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=500, silence_thresh=-50)

    trimmed_audio = AudioSegment.empty()
    for start, end in nonsilent_ranges:
        trimmed_audio += audio[start:end]

    clipped_audio = trimmed_audio[:duration]
    return clipped_audio.export(format="wav").read()

In [11]:
import os
import pandas as pd
from pydub import AudioSegment

data = []
error = []
start = 500

i = 0
result = result_part

for bird_id,sound_ids in result.items():#len(result):9684
    # if i > 500:
    print('\r%d / %d' %(i, len(result)), end='')
    for sound_id in sound_ids:
        try:
            file_path = os.path.join("D:","xeno_data",bird_id, sound_id)
            audio = AudioSegment.from_mp3(file_path)
            wav_data = audio.export(format="wav").read()
            # wav_data = process_audio(file_path)
            wav_dict = {"bytes":wav_data,"path":None}
            data.append([sound_id.replace(".mp3",""),wav_dict,bird_id])
        except:
            error.append(file_path)
    #     if i % 100 == 0 or i == len(result)-1:
    #         # DataFrameを作成
    #         data_df = pd.DataFrame(data, columns=["s_id","audio","description"])
    #         del data
    #         data_df.set_index("s_id", inplace=True)
    #         data_df.index.name = None
    #         try:
    #             data_df.to_parquet('D:/wav_model_data/all_xeno_~'+str(8900+i)+'.parquet')
    #         except:
    #             data_df.to_parquet('./wav_model_data/all_xeno_~'+str(8900+i)+'.parquet')
    #         # del data_df
    #         data = []
    i = i + 1
# 5600 / 9684

1 / 2

In [16]:
for i in range(len(data)):
    print(i)

0
1
2
3
4
5


In [15]:
print(len(data))

6


In [ ]:
data[0][1]

KeyError: slice(None, 100, None)

In [ ]:
# origilal

import os
import pandas as pd
from pydub import AudioSegment

data = []
error = []
start = 500

i = 0

for bird_id,sound_ids in result.items():#len(result):9684
    if i > 500:
        print('\r8900 + %d / %d' %(i, len(result)), end='')
        for sound_id in sound_ids:
            try:
                file_path = os.path.join("xeno_data",bird_id, sound_id)
                audio = AudioSegment.from_mp3(file_path)
                wav_data = audio.export(format="wav").read()
                wav_dict = {"bytes":wav_data,"path":None}
                data.append([sound_id.replace(".mp3",""),wav_dict,bird_id])
            except:
                error.append(file_path)
        if i % 100 == 0 or i == len(result)-1:
            # DataFrameを作成
            data_df = pd.DataFrame(data, columns=["s_id","audio","description"])
            del data
            data_df.set_index("s_id", inplace=True)
            data_df.index.name = None
            try:
                data_df.to_parquet('D:/wav_model_data/all_xeno_~'+str(8900+i)+'.parquet')
            except:
                data_df.to_parquet('./wav_model_data/all_xeno_~'+str(8900+i)+'.parquet')
            # del data_df
            data = []
    i = i + 1
# 5600 / 9684

In [47]:
data_df

,audio,description


In [48]:
error

['xeno_data\\Q1091288\\320216.mp3',
 'xeno_data\\Q1091288\\320217.mp3',
 'xeno_data\\Q1091288\\320218.mp3',
 'xeno_data\\Q1091288\\320219.mp3',
 'xeno_data\\Q1091288\\320220.mp3',
 'xeno_data\\Q1091288\\320221.mp3',
 'xeno_data\\Q1091288\\320222.mp3',
 'xeno_data\\Q1091293\\111180.mp3',
 'xeno_data\\Q1091293\\111181.mp3',
 'xeno_data\\Q1091293\\125325.mp3',
 'xeno_data\\Q1091293\\125326.mp3',
 'xeno_data\\Q1091293\\125327.mp3',
 'xeno_data\\Q1091293\\172543.mp3',
 'xeno_data\\Q1091293\\175605.mp3',
 'xeno_data\\Q1091293\\200830.mp3',
 'xeno_data\\Q1091293\\200933.mp3',
 'xeno_data\\Q1091293\\200935.mp3',
 'xeno_data\\Q1091293\\26507.mp3',
 'xeno_data\\Q1091293\\27618.mp3',
 'xeno_data\\Q1091293\\40867.mp3',
 'xeno_data\\Q1091293\\40868.mp3',
 'xeno_data\\Q1091293\\469727.mp3',
 'xeno_data\\Q1091293\\596482.mp3',
 'xeno_data\\Q1091293\\596483.mp3',
 'xeno_data\\Q1091293\\599956.mp3',
 'xeno_data\\Q1091293\\604609.mp3',
 'xeno_data\\Q1091293\\61383.mp3',
 'xeno_data\\Q1091293\\653917.mp3

In [8]:
data_df.to_parquet('./wav_model_data/all_xeno_~'+str(6300+i)+'.parquet')
# del data_df
# data = []

In [ ]:
# del data_df
# data = []

In [ ]:
len(data_df)

In [ ]:
start = 0

In [10]:
file_list = []
j = 0
start = 3600
for bird_id,sound_ids in result.items():#len(result):9684
    print('\r%d / %d' %(j, 5885), end='')
    if j > start and j < 5885:
        file_list.append(bird_id)
    j = j + 1

5884 / 5885

In [11]:
len(file_list)

2284

In [12]:
import os
# リネーム処理
for file_name in file_list:
    old_path = os.path.join("xeno_data",file_name)
    new_path = os.path.join("done",file_name)
    if os.path.exists(old_path):  # 元ファイルが存在するか確認
        os.rename(old_path, new_path)
    else:
        print(f"File '{old_name}' does not exist.")

In [29]:
# checks

In [10]:
import numpy as np

In [30]:
i

9684

In [ ]:
num_splits = 3
dfs = np.array_split(data_df, num_splits)

# 結果表示
for n, split_df in enumerate(dfs):
    print(f"DataFrame {n+1}:\n{split_df}\n")
    split_df.to_parquet('wav_model_data/all_xeno_~'+str(i)+'-'+str(n+1)+'.parquet')

C:\Users\momo1\Projects\env_webscraper\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
# del data

In [ ]:
# data_df.to_parquet('wav_model_data/all_xeno_~'+str(i)+'.parquet')

In [ ]:
len(data_df)

In [72]:
print(data_df)

                                                    audio description
204670  {'bytes': b'RIFF(\x02\xaf\x00WAVEfmt \x10\x00\...    Q1000262
280373  {'bytes': b'RIFF$\xc0*\x00WAVEfmt \x10\x00\x00...    Q1000262
280375  {'bytes': b'RIFF$b=\x00WAVEfmt \x10\x00\x00\x0...    Q1000262
165127  {'bytes': b'RIFF$\x0f9\x00WAVEfmt \x10\x00\x00...    Q1000977
184094  {'bytes': b'RIFF\xe4\x81h\x00WAVEfmt \x10\x00\...    Q1000977
184095  {'bytes': b'RIFF\xbcF\x17\x00WAVEfmt \x10\x00\...    Q1000977
246707  {'bytes': b'RIFF\xc4\x9bc\x00WAVEfmt \x10\x00\...    Q1001580
246708  {'bytes': b'RIFF4\x8aT\x00WAVEfmt \x10\x00\x00...    Q1001580
24686   {'bytes': b'RIFF$\xed!\x00WAVEfmt \x10\x00\x00...    Q1001580


In [73]:
data_df.to_parquet('wav_model_data/all_xeno.parquet')

In [100]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3, shuffle=True, random_state=101)

for i, (train_i, test_i) in enumerate(kf.split(data_df)):
    train_df = data_df.iloc[train_i]
    test_df = data_df.iloc[test_i]
    train_df.to_parquet(f'wav_model_data/train_{i+1}.parquet')
    test_df.to_parquet(f'wav_model_data/test_{i+1}.parquet')

In [101]:
# 以降データチェック

In [102]:
df = pd.read_parquet('wav_model_data/train_3.parquet')#10313

In [103]:
len(data_df)

9

In [104]:
data_df

,audio,description
204670,{'bytes': b'RIFF(\x02\xaf\x00WAVEfmt \x10\x00\...,Q1000262
280373,{'bytes': b'RIFF$\xc0*\x00WAVEfmt \x10\x00\x00...,Q1000262
280375,{'bytes': b'RIFF$b=\x00WAVEfmt \x10\x00\x00\x0...,Q1000262
165127,{'bytes': b'RIFF$\x0f9\x00WAVEfmt \x10\x00\x00...,Q1000977
184094,{'bytes': b'RIFF\xe4\x81h\x00WAVEfmt \x10\x00\...,Q1000977
184095,{'bytes': b'RIFF\xbcF\x17\x00WAVEfmt \x10\x00\...,Q1000977
246707,{'bytes': b'RIFF\xc4\x9bc\x00WAVEfmt \x10\x00\...,Q1001580
246708,{'bytes': b'RIFF4\x8aT\x00WAVEfmt \x10\x00\x00...,Q1001580
24686,{'bytes': b'RIFF$\xed!\x00WAVEfmt \x10\x00\x00...,Q1001580


In [105]:
len(df)

6

In [106]:
df

,audio,description
204670,{'bytes': b'RIFF(\x02\xaf\x00WAVEfmt \x10\x00\...,Q1000262
280375,{'bytes': b'RIFF$b=\x00WAVEfmt \x10\x00\x00\x0...,Q1000262
165127,{'bytes': b'RIFF$\x0f9\x00WAVEfmt \x10\x00\x00...,Q1000977
184094,{'bytes': b'RIFF\xe4\x81h\x00WAVEfmt \x10\x00\...,Q1000977
246708,{'bytes': b'RIFF4\x8aT\x00WAVEfmt \x10\x00\x00...,Q1001580
24686,{'bytes': b'RIFF$\xed!\x00WAVEfmt \x10\x00\x00...,Q1001580
